In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

#from mtflearn import ZPs

In [2]:
from mtflearn.features import autocorrelation, radial_profile, get_characteristic_length, get_characteristic_length_fft
from mtflearn.utils import normalize_image

In [3]:
from skimage.transform import resize

## Load image

In [4]:
filename = 'data\\monolayer_MoSe2_80K.npy'
img = np.load(filename)[0:800, 0:800]
img = resize(img, (512, 512))
img = normalize_image(img, 0, 1)

## Method 1: get lattice constant from Fourier space

In [7]:
s = get_characteristic_length_fft(img)
a1 = 2/np.sqrt(3) * s
print('Lattice constant is {} pixels.'.format(a1))

Lattice constant is 28.15269884048461 pixels.


## Method 2: get lattice constant from raidal distribution function

In [8]:
a2 = get_characteristic_length(img)

print('Lattice constant is {} pixels.'.format(a2))

Lattice constant is 27 pixels.
